In [3]:
import os
import sys

path_project = os.path.abspath(os.path.join(os.getcwd(), ".."))
if path_project not in set(sys.path):
    sys.path.append(path_project)

In [4]:
import itertools
import matplotlib.pyplot as plt
import polars as pl
from scipy.sparse import csr_matrix
from scripts.als import ALS
import json

# Recommender AI System

This notebook shows all the steps that we will take to build our machine learning model. In this case, we decided to use the library `Polars` for manipulating the training dataset, because this has a better performance when working with large datasets compared to `Pandas`. This is a big help for us, because we don't have the best hardware for this task, so the most efficient tools are our friends.    

Basically, the goal is to build a recommender system that is capable to recommend 5 different products to a customer by checking its purchase history. The columns of the training dataset are the following:
* Session ID
* Date of interaction
* Interaction timestamp
* User ID
* User's country
* Partnumber: The id of the product with which the interaction occurred.
* device_type: Type of device used.
* pagetype: Type of page where the interaction occurred within the e-commerce site.
* add_to_cart: Boolean indicating if the interaction was adding to the card.

The approach to create a recommender system is based on **collaborative filtering**. This technique consists of making assumptions about a user based on the information from other users who have similar behaviors when purchasing or interacting with items. This data can reveal **latent patterns** that represent both the preferences of clients and the characteristics of the products/items (Hu, Y. et all, 2008, p. 1).

To apply this concept mathematically, we need to use matrix factorization, specifically **ALS (Alternating Least Squares)**. First, we have to create a matrix of interactions between products and clients. We face the challenge that there are null values in the user_id field, but this information is really important because it reveals which products are viewed together, add-to-cart patterns, and other valuable behavioral insights. Therefore, we have decided to include these records (85% of the data) by using the `session_id` as a pseudo-user identifier.

In [5]:
# Load data
df_train_data: pl.LazyFrame = pl.scan_parquet("../data/train.parquet") # using Lazy API

We will assign a pseudo-user to those record that lack a user id based on their session id's:

In [6]:
# Expression to modify the user_id
unified_user= (
    pl.when( pl.col('user_id').is_not_null() )
        .then( pl.concat_str( [pl.lit("u_"), pl.col("user_id").cast(pl.Utf8)] ) )
        .otherwise( pl.concat_str( [pl.lit("s_"), pl.col("session_id").cast(pl.Utf8) ]) )
        .alias("unified_user_id")
)

# Batch processing:
CHUNK = 1_000_000
N: int = df_train_data.select( pl.len() ).collect().item() # Nrows

df_unified : pl.LazyFrame = pl.concat(
    df_train_data.slice(offset, CHUNK).with_columns(unified_user).collect()
    for offset in range(0, N, CHUNK)
).lazy()

The problem is that this interaction matrix is sparse, because it is most likely that a client doesn't interact with all the products in the catalog. Therefore, a solution is to decompose it into two dense matrices: $X$ and $Y$:

$$R_{m \times n} = X_{m \times f} \times Y_{f \times n}^T$$

Where:

* $R \rightarrow$ Interaction matrix (users $\times$ items)
* $X$ and $Y$ are the dense matrices, where $X$ represents user latent factors and $Y$ represents item latent factors
* $f \rightarrow$ Represents the number of latent factors, whose value is much lower than $m$ and $n$

---

In e-commerce, we rarely have explicit ratings. We instead have implicit feedback, meaning the frequency of actions that suggest interest but don't directly tell us how much a user likes something (Hu, Y. et all, 2008, p. 2). You can think of it this way:

* View a product page (mild interest)
* View it multiple times (stronger interest)
* Add it to cart (very strong interest)
* Purchase it (maximum interest)

To translate these actions to numbers in order to ALS algorithm can understand, we use the something called *confidence*. They will represent the values in the table of interactions and the way to compute them as follows:$$c_{ui} = 1 + \text{view count} + 10\cdot\text{cart count}$$
Where:
* View count: How many times the user $u$ has seen the product or item $i$.
* Cart count: How many times the user $u$ has added the product $i$ to a cart. This signal is more stronger than a view, so it's weighted 10 times more.

In [7]:
# Compute the view and car count for each pair (user, product):
df_interactions = df_unified.group_by(["unified_user_id", "partnumber"]).agg([
            pl.len().alias("view_count"),
            pl.col("add_to_cart").sum().alias("cart_count").fill_null(0)
])

# Calculate confidence scores (implicit feedback)
df_interactions = df_interactions.with_columns([
    (1 + pl.col("view_count") + 10 * pl.col("cart_count")).alias("confidence")
])

Once calculated the confidence values, we can create the sparse matrix $R$ (`df_interactions`) as follows:

In [8]:
user_mapping = (
    df_interactions.select("unified_user_id")
                    .unique()
                    .sort("unified_user_id")
                    .with_row_index("user_id")
)

item_mapping = (
    df_interactions.select('partnumber')
                    .unique()
                    .sort('partnumber')
                    .with_row_index('item_id')
)

# Join to get indices
df_interactions = (
    df_interactions
    .join(user_mapping, on='unified_user_id')
    .join(item_mapping, on='partnumber')
).collect()

# Sparse matrix
csr_interactions = csr_matrix(
    ( # (data, (row_idx, col_idx))
        df_interactions["confidence"].to_numpy(),
        ( df_interactions["user_id"].to_numpy(),
          df_interactions["item_id"].to_numpy() ) 
    ), shape = ( user_mapping.select(pl.len()).collect().item(), item_mapping.select(pl.len()).collect().item() )
)

It's important to understand the loss function that we are trying to minimize when using the algoritm ALS:

$$L = \sum_{(u,i) \in \Omega} c_{ui}(r_{ui} - x_u^T y_i)^2 + \lambda\left(\sum_u ||x_u||^2 + \sum_i ||y_i||^2\right)$$

Think of this equation as a way to measure "how wrong" our recommendations are, and our goal is to make this number as small as possible. Let's understand each part:

* $r_{ui} - x_u^T y_i$ represents the difference between what actually happened (did the user interact with the item?) and what our model predicts.
* Think of $\hat{p}_{ui} = x_u^T y_i$ as a "compatibility score" (or preference) between user $u$ and item $i$. The "magic" behind the dot product is that is mathematical way of measuring similarity - the higher the number, the better the match:$$x\cdot y^t = |x||y|\cos(\theta) $$
* $c_{ui}$ acts like a "trust multiplier" - some interactions are more reliable than others.
* $\lambda\left(\sum_u ||x_u||^2 + \sum_i ||y_i||^2\right)$ prevents the model from becoming too complex.



<u>Parameters explanation:</u>

* $c_{ui}$: Confidence level of user $u$ for product/item $i$. This is crucial for an implicit feedback system.
* $r_{ui}$: The input data associate between user $u$ and item $i$.
* $x_u$: Latent factor vector representing user $u$
* $y_i$: Latent factor vector representing item $i$
* $\lambda$: Regularization parameter to avoid overfitting.
* $\Omega$: Set of observed interactions between users and products.

---

### Algorithm

You may be wondering how this algorithm works. Let's explain:

1. Initialization: $X$ and $Y$ are initialized randomly. We also set the key parameters that define the training stage: confidence multiplier ($\alpha$), regularization ($\lambda$), and number of factors ($f$).
2. Fix $Y$, solve for $X$: Due to ALS nature, you can only optimize one matrix at a time, keeping the other one fixed. For each user, we solve the following equation:
$$x_u = (Y^T C^u Y + \lambda I)^{-1} Y^T C^u r^u$$
3. Fix $X$, solve for $Y$: The process is the same as step 2, but now we optimize the item factors while keeping user factors fixed.
4. Repeat: Continue alternating between steps 2 and 3 until the algorithm converges or the maximum number of iterations is reached. Each step (i.e. recomputing user-factors and item-factors) is guaranteed to lower the value of the cost function.




<u>Key Parameters and their impact:</u>

According to Hu Y. et all (2008):

* Number of Factors ($f$): This determines the dimensionality of the latent space. Think of it as the number of hidden "concepts" or "topics" that you want to capture about user preferences and item characteristics.
    * Sweet spot: 20-200 for large volumes of data

* Regularization Parameter ($\lambda$): Controls how much we penalize large factor values to prevent overfitting.

    * Sweet spot: 0.1-1.0

* Confidence Multiplier ($\alpha$): Determines how much more we trust frequent interactions compared to single interactions. We won't use it, because the we have defined the way to calculate the confidence values.

    * Typical range: 1-40

* Number of Iterations: The balance between convergence and overfitting.

    * Typical range: 15-30
 

---

### Training the model

* Use different number of factors.
* Better results are achieved when the model is regularized (Use different lambdas)

We will use different settings to determine the model with the best performance during the validation stage. We will increase the number of latent factors to see how the model behaves, check whether the performance gets worse when relaxing the regularization parameter. We will also increase our confidence in frequent interactions and try with different number of iterations to study how the model behaves based on this variable.


In [ ]:
df_results = []

# Set parameters
param_grid = {
    "factors": [20, 70, 150],
    "regularization": [0.1, 1],
    "iterations": [15, 20]}

# json file provided by INDITEX for performing the validation stage.
with open("../data/example_predictions_3.json") as f:
    predictions = json.load(f)
predictions = predictions["target"]

# Let's start training the models
models = []
for setting in itertools.product(*param_grid.values()):
    params = dict( zip(param_grid.keys(), setting) )
    df_results.append(params)

    # Build and create the model.
    model = ALS(**params)
    model = model.train(csr_interactions)
    models.append(model)


# VALIDATION STAGE

# The outputs of the models have to be translated to real items ids to measure their accuracies.
translate_item_id = lambda item_id: (
    item_mapping
        .filter(plt.col("item_id") == item_id)
        .select("partnumber")
        .collect()
        .item()
) 

i = 0
for user_id, items_id in predictions.items():
    items_id = set(items_id) # No matter the order

    # Translate the user_id by using the user_mapping
    user_id = (
        user_mapping
            .filter(pl.col("unified_user_id") == f"u_{user_id}") 
            .select("user_id")
            .collect()
            .item()
    )

    # Get the outputs
    outputs = models[i].recommend(user_id, csr_interactions[user_id], N=5, filter_already_liked_items=True)
    # It's necessary to mapping to original item ids
    items_outputs = {translate_item_id(item_id) for item_id in outputs[0]}

    df_results[i]["accuracy"] = len(items_id & items_outputs) / 5
    
    i += 1

# Performance Summary:
for i in range(3):
    print(f"Accuracy of model {i}: {df_results[i]["accuracy"]: .2f}") 

# Save results:
pl.DataFrame(df_results).write_csv("../data/models_accuracy.csv")

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
als = ALS()
model = model.train(csr_interactions)

In [ ]:
# You have to use the id used in csr matrix.
model.recommend(85, csr_interactions[85], N=5, filter_already_liked_items=True)

---

## Bibliography

* Hu, Y., Koren, Y., & Volinsky, C. (2008). Collaborative filtering for implicit feedback datasets. In 2008 Eighth IEEE International Conference on Data Mining (pp. 263-272).